In [157]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import cv2
import xml.etree.ElementTree as ET
import os
import matplotlib.pyplot as plt
import numpy as np
import shapely
from shapely.ops import unary_union
from shapely.geometry import LineString
from shapely.ops import linemerge
from statistics import mean, median

from shapely.geometry import LineString



In [158]:
images_path = f'/home/ohm/FiFi/FiFi_Dataset/Selected_Pictues_2.0/All/'

# Parse the XML file
xml_file = '/home/ohm/Documents/HTWKRobots/hackingweekend/naoTeamRepo/firmware_5.0/vision/demo/annotations.xml'
tree = ET.parse(xml_file)
root = tree.getroot()

xml_file_compare = '/home/ohm/PycharmProjects/FiFi/annotations/annotations_unchanged.xml'
tree_compare = ET.parse(xml_file_compare)
root_compare = tree_compare.getroot()

buffer_size = 5

# Function to parse points from a string
def parse_points(points_str):
    points = []
    for point in points_str.split(';'):
        x, y = point.split(',')
        points.append((float(x), float(y)))
    points = np.array(points, dtype=np.int32)
    return points



In [159]:
import numpy as np
from matplotlib.path import Path
from matplotlib.patches import PathPatch
from matplotlib.collections import PatchCollection


def plot_polygon(poly, img, line_color=(0, 255, 0), line_thickness=2):
    # Get the exterior coordinates of the polygon
    exterior_coords = np.array(poly.exterior.coords).astype(int)

    # Draw the polygon on the image
    cv2.polylines(img, [exterior_coords], isClosed=True, color=line_color, thickness=line_thickness)

def plot_img(element, image):

    for polygon in element.iter('polygon'):
        str_point = polygon.get('points')
        pointss = parse_points(str_point)
        pointss = np.append(pointss, [pointss[0]], axis=0)
        for point_idx, ele in np.ndenumerate(pointss):
            point_idx = point_idx[0]
            plot_polygon(LineString([pointss[point_idx], pointss[(point_idx + 1) % len(pointss)]]).buffer(buffer_size+4), image, line_thickness=1)

    for line in element.iter('line'):
        str_point = line.get('points')
        pointss = parse_points(str_point)
        # draw the line on the image
        cv2.line(image, tuple(pointss[0]), tuple(pointss[1]), (0, 0, 255), 2)

    print(element.attrib['name'])




    return image



In [160]:
def calc_metrics(element):
    ground_truth = []
    detector_output = []
    iou = 0

    for line_element in element.iter('line'):
        points_str = line_element.get('points')
        points = parse_points(points_str)
        detector_output.append(LineString(points).buffer(buffer_size))

    for polygon_element in element.iter('polygon'):
        points_str = polygon_element.get('points')
        points = parse_points(points_str)
        points = np.append(points, [points[0]], axis=0)
        for point_idx, ele in np.ndenumerate(points):
            point_idx = point_idx[0]
            ground_truth.append(LineString([points[point_idx], points[(point_idx + 1) % len(points)]]).buffer(buffer_size))


    union_all_poly = unary_union(ground_truth)
    union_all_line = unary_union(detector_output)

    intersection = union_all_poly.intersection(union_all_line)

    union = union_all_poly.union(union_all_line)

    iou = intersection.area / union.area

    # calculate false positives area
    # get the area where the detector output is not in the ground truth
    diff = calc_false_positives(element)

    return diff, iou

def calc_false_positives(ele):
    ground_truth = []
    detector_output = []

    for line_element in ele.iter('line'):
        points_str = line_element.get('points')
        points = parse_points(points_str)
        detector_output.append(LineString(points))

    for polygon_element in ele.iter('polygon'):
        points_str = polygon_element.get('points')
        points = parse_points(points_str)
        points = np.append(points, [points[0]], axis=0)
        for point_idx, ele in np.ndenumerate(points):
            point_idx = point_idx[0]
            ground_truth.append(LineString([points[point_idx], points[(point_idx + 1) % len(points)]]).buffer(buffer_size+4))

    # calculate the lenght of the detector_output lines that are not in the ground_truth polygons
    diff = unary_union(detector_output).difference(unary_union(ground_truth))

    return diff.length

# a function that calculates the true_positives for a given element
def calc_true_positives(element):
    ground_truth = []
    detector_output = []

    for line_element in element.iter('line'):
        points_str = line_element.get('points')
        points = parse_points(points_str)
        detector_output.append(LineString(points))

    for polygon_element in element.iter('polygon'):
        points_str = polygon_element.get('points')
        points = parse_points(points_str)
        points = np.append(points, [points[0]], axis=0)
        for point_idx, ele in np.ndenumerate(points):
            point_idx = point_idx[0]
            ground_truth.append(LineString([points[point_idx], points[(point_idx + 1) % len(points)]]).buffer(buffer_size+4))

    # calculate the lenght of the detector_output lines that are in the ground_truth polygons
    intersection = unary_union(detector_output).intersection(unary_union(ground_truth))

    return intersection.length

# a function to calculate the false negatives
def calc_false_negatives(element):
    ground_truth = []
    detector_output = []

    for line_element in element.iter('line'):
        points_str = line_element.get('points')
        points = parse_points(points_str)
        detector_output.append(LineString(points))

    for polygon_element in element.iter('polygon'):
        points_str = polygon_element.get('points')
        points = parse_points(points_str)
        points = np.append(points, [points[0]], axis=0)
        for point_idx, ele in np.ndenumerate(points):
            point_idx = point_idx[0]
            ground_truth.append(LineString([points[point_idx], points[(point_idx + 1) % len(points)]]).buffer(buffer_size+4))

    # calculate the length of the ground_truth polygons that are not in the detector_output lines
    diff = unary_union(ground_truth).difference(unary_union(detector_output))

    return diff.length


def calc_metrics_for_all_images(xml_root, xml_root_compare_plot=None):
    iou_array = []
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    for image_element in xml_root.iter('image'):
        image_name = image_element.get('name')
        image_path = images_path + image_name

        img = cv2.imread(image_path)

        if not os.path.exists(image_path):
            continue

        false_positive, iou = calc_metrics(image_element)
        iou_array.append(iou)

        false_positives += false_positive

        true_positive = calc_true_positives(image_element)
        true_positives += true_positive

        false_negative = calc_false_negatives(image_element)
        false_negatives += false_negative

        recall = true_positive / (true_positive + false_negative)
        if true_positive + false_positive == 0:
            precision = 0
        else:
            precision = true_positive / (true_positive + false_positive)


        # check if xml_root_compare_plot is None
        if xml_root_compare_plot is not None:
            image_element_compare = None
            for ele in xml_root_compare_plot.iter('image'):
                if ele.attrib['name'] == image_element.attrib['name']:
                    image_element_compare = ele
                    compare_false_positive, compare_iou = calc_metrics(image_element_compare)
                    compare_true_positive = calc_true_positives(image_element_compare)
                    comapre_false_negative = calc_false_negatives(image_element_compare)
                    compare_recall = compare_true_positive / (compare_true_positive + comapre_false_negative)
                    if compare_true_positive + compare_false_positive == 0:
                        compare_precision = 0
                    else:
                        compare_precision = compare_true_positive / (compare_true_positive + compare_false_positive)

                    path = f"/home/ohm/PycharmProjects/FiFi/compare/recall/worse/{image_name}"
                    if compare_recall < recall:
                        path = f"/home/ohm/PycharmProjects/FiFi/compare/recall/better/{image_name}"

                    if compare_recall == recall:
                        path = f"/home/ohm/PycharmProjects/FiFi/compare/recall/same/{image_name}"

                    compare_img = img
                    compare_img = plot_img(image_element_compare, compare_img)
                    # write precison and recall on the image
                    # save compare image as np array
                    compare_img = np.array(compare_img)
                    cv2.putText(compare_img, f"Old precision: {compare_precision:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    cv2.putText(compare_img, f"Old recall: {compare_recall:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

                    img = plot_img(image_element, img)
                    # write precison and recall on the image
                    cv2.putText(img, f"New precision: {precision:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    cv2.putText(img, f"New recall: {recall:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

                    # combine two images into one
                    img = np.concatenate((img, compare_img), axis=1)

                    # save the image
                    cv2.imwrite(path, img)





    # calculate the precision
    precision = true_positives / (true_positives + false_positives)
    # calulate recall
    recall = true_positives / (true_positives + false_negatives)

    # calculate f1 score with more weight on precision
    beta = 0.5
    f1 = (1 + beta**2) * (precision * recall) / ((beta**2 * precision) + recall)


    return precision, recall, iou_array, f1

In [161]:
precision, recall, ious, f1 = calc_metrics_for_all_images(root, xml_root_compare_plot=root_compare)
precision_compare, recall_comapre, ious_compare, f1_compare = calc_metrics_for_all_images(root_compare)

GO2018G11B_GLaDOS_00125368_L.png
GO2018G11B_GLaDOS_00125368_L.png
GO2018G11B_GLaDOS_00375502_L.png
GO2018G11B_GLaDOS_00375502_L.png
GO2018G11B_GLaDOS_00462305_L.png
GO2018G11B_GLaDOS_00462305_L.png
GO2018G11B_GLaDOS_00466858_L.png
GO2018G11B_GLaDOS_00466858_L.png
GO2018G11B_GLaDOS_00468562_L.png
GO2018G11B_GLaDOS_00468562_L.png
GO2018G11B_GLaDOS_00470364_L.png
GO2018G11B_GLaDOS_00470364_L.png
GO2018G11B_GLaDOS_00473779_L.png
GO2018G11B_GLaDOS_00473779_L.png
GO2018G11B_GLaDOS_00477293_L.png
GO2018G11B_GLaDOS_00477293_L.png
GO2018G11B_GLaDOS_00483573_L.png
GO2018G11B_GLaDOS_00483573_L.png
GO2018G11B_GLaDOS_00554398_L.png
GO2018G11B_GLaDOS_00554398_L.png
GO2018G11B_GLaDOS_00556479_L.png
GO2018G11B_GLaDOS_00556479_L.png
GO2018G11B_GLaDOS_00560016_L.png
GO2018G11B_GLaDOS_00560016_L.png
GO2018G11B_GLaDOS_00563871_L.png
GO2018G11B_GLaDOS_00563871_L.png
GO2018G11B_GLaDOS_00565701_L.png
GO2018G11B_GLaDOS_00565701_L.png
GO2018G11B_GLaDOS_00567429_L.png
GO2018G11B_GLaDOS_00567429_L.png
GO2018G11B

In [162]:
def print_metrics(precision, recall, iou_array, f1, name=""):
    print(f"Metrics for: {name}")
    print(f"F1 score: {f1}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"Mean IOU {mean(iou_array)}")
    print(f"Median IOU {median(iou_array)}")
    print(f"Imges with no detection: {len([iou for iou in iou_array if iou < 0.001])}")
    print(f"Imges with no detection: {len(iou_array)}")
    #plt.hist(iou_array, bins='auto')
    plt.show()

In [164]:
print_metrics(precision, recall, ious, f1, name=xml_file)
print_metrics(precision_compare, recall_comapre, ious_compare, f1_compare, name=xml_file_compare)

Metrics for: /home/ohm/Documents/HTWKRobots/hackingweekend/naoTeamRepo/firmware_5.0/vision/demo/annotations.xml
F1 score: 0.6175455184679326
Precision: 0.9504849566404947
Recall: 0.2571890022672699
Mean IOU 0.26875292406540197
Median IOU 0.24644715319283578
Imges with no detection: 45
Imges with no detection: 160
Metrics for: /home/ohm/PycharmProjects/FiFi/annotations/annotations_unchanged.xml
F1 score: 0.366758015164733
Precision: 0.958998140455674
Recall: 0.10568648604630498
Mean IOU 0.13409544400570192
Median IOU 0.0
Imges with no detection: 101
Imges with no detection: 160
